1. Data Preparation

In [ ]:
import os
import zipfile
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from torch.optim import Adam
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [ ]:
# Path to your zip file
zip_path = os.path.expanduser('~/Downloads/archive(1).zip')
extract_dir = './flickr8k_data'

# Extract only if not already done
if not os.path.exists(extract_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

print("Extracted to:", extract_dir)
print("Contents:", os.listdir(extract_dir))

In [ ]:
extract_dir = '/Users/sfowler14/Downloads/archive'
print("Extracted to:", extract_dir)
print("Contents:", os.listdir(extract_dir))

In [ ]:
captions_path = os.path.join(extract_dir, 'captions.txt')
df = pd.read_csv(captions_path)

print(df.head())
print(f"\nNumber of unique images: {df['image'].nunique()}")

2. Image Feature Extraction

In [ ]:
# --- Image transform (standard ImageNet normalization) ---
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Dataset class for images ---
class FlickrImageDataset(Dataset):
    def __init__(self, image_dir, image_filenames, transform=None):
        self.image_dir = image_dir
        self.image_filenames = list(image_filenames)
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = self.image_filenames[idx]
        path = os.path.join(self.image_dir, img_name)
        # wrap in try-except to catch corrupt images
        try:
            image = Image.open(path).convert("RGB")
        except Exception as e:
            # if image fails to open, create a black image instead and log
            print(f"Failed to open {path}: {e}")
            image = Image.new('RGB', (224,224))
        if self.transform:
            image = self.transform(image)
        return image, img_name

In [ ]:
# --- Model setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*(list(resnet.children())[:-1]))  # remove FC layer
resnet.to(device)
resnet.eval()

# --- Create dataset and dataloader ---
image_dir = os.path.join(extract_dir, "Images")
unique_images = df["image"].unique()
image_dataset = FlickrImageDataset(image_dir, unique_images, transform)
image_loader = DataLoader(image_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# test model forward
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*(list(resnet.children())[:-1])).to(device).eval()

In [ ]:
image_filenames = sorted(os.listdir(image_dir))   # better to use df['image'].unique() if you want same order
dataset = FlickrImageDataset(image_dir, image_filenames, transform)
loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)  # num_workers=0 is safest

image_features_list = []
image_names = []

In [ ]:
with torch.no_grad():
    for imgs, names in tqdm(loader, desc="Extracting image features"):
        imgs = imgs.to(device)
        feats = resnet(imgs)               # (B, 2048, 1, 1)
        feats = feats.view(feats.size(0), -1)  # (B, 2048)
        image_features_list.append(feats.cpu())
        image_names.extend(names)

image_features = torch.cat(image_features_list, dim=0).numpy()
print("Done. image_features shape:", image_features.shape)

3. Caption Feature Extraction

In [ ]:
from transformers import BertTokenizer, BertModel

# --- Device and model setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')
bert.to(device)
bert.eval()

# --- Dataset for captions ---
class FlickrCaptionDataset(Dataset):
    def __init__(self, captions):
        self.captions = captions
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        text = self.captions[idx]
        # Return plain text; we’ll tokenize in collate_fn for batching
        return text

# --- Custom collate_fn to batch tokenize ---
def collate_fn(batch_texts):
    return tokenizer(batch_texts, return_tensors='pt',
                     truncation=True, padding=True, max_length=64)


In [ ]:
# --- Create dataset and dataloader ---
caption_dataset = FlickrCaptionDataset(df["caption"].tolist())
caption_loader = DataLoader(
    caption_dataset,
    batch_size=32,         
    shuffle=False,
    num_workers=0,         
    collate_fn=collate_fn  
)

In [ ]:
# --- Extract features ---
caption_features = []

with torch.no_grad():
    for batch in tqdm(caption_loader, desc="Extracting caption features"):
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = bert(**inputs)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]  # (B, 768)
        caption_features.append(cls_embeddings.cpu())

caption_features = torch.cat(caption_features, dim=0).numpy()

print("Caption features shape:", caption_features.shape)

4. Data Splitting

In [ ]:
# Map image filename to its index
image_to_idx = {name: i for i, name in enumerate(image_names)}

# For each caption row, find which image it corresponds to
caption_to_image_idx = df["image"].map(image_to_idx).values

print("caption_to_image_idx shape:", caption_to_image_idx.shape)
print("Example mapping:", list(zip(df['caption'][:3], caption_to_image_idx[:3])))

In [ ]:
np.save("flickr8k_image_features.npy", image_features)
np.save("flickr8k_caption_features.npy", caption_features)
np.save("flickr8k_caption_to_image.npy", caption_to_image_idx)
np.save("flickr8k_image_names.npy", np.array(image_names))

In [ ]:
# Split on images
n_images = len(image_names)
indices = np.arange(n_images)

In [ ]:
# 70/15/15 split
train_idx, temp_idx = train_test_split(indices, test_size=0.30, random_state=42, shuffle=True)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42, shuffle=True)
print(f"Train images: {len(train_idx)}, Val: {len(val_idx)}, Test: {len(test_idx)}")

In [ ]:
# Create masks for captions 
caption_to_image_idx = caption_to_image_idx.astype(int)

train_mask = np.isin(caption_to_image_idx, train_idx)
val_mask = np.isin(caption_to_image_idx, val_idx)
test_mask = np.isin(caption_to_image_idx, test_idx)

# Split image features 
image_train = image_features[train_idx]
image_val   = image_features[val_idx]
image_test  = image_features[test_idx]

# Split captions (and keep their alignment)
caption_train = caption_features[train_mask]
caption_val   = caption_features[val_mask]
caption_test  = caption_features[test_mask]

In [ ]:
# Link captions to local image indices within each split
def remap_caption_indices(global_indices, split_indices):
    """
    Convert global image indices in caption_to_image_idx to 0..len(split_indices)-1 within that split.
    """
    mapping = {g: i for i, g in enumerate(split_indices)}
    return np.array([mapping[i] for i in global_indices if i in mapping])

In [ ]:
caption_to_train_img = remap_caption_indices(caption_to_image_idx[train_mask], train_idx)
caption_to_val_img   = remap_caption_indices(caption_to_image_idx[val_mask], val_idx)
caption_to_test_img  = remap_caption_indices(caption_to_image_idx[test_mask], test_idx)

In [ ]:
# Sanity check
print("Train split shapes:")
print("  Image features:", image_train.shape)
print("  Caption features:", caption_train.shape)
print("  Caption→Image indices:", caption_to_train_img.shape)

In [ ]:
np.save("train_image_features.npy", image_train)
np.save("val_image_features.npy", image_val)
np.save("test_image_features.npy", image_test)

np.save("train_caption_features.npy", caption_train)
np.save("val_caption_features.npy", caption_val)
np.save("test_caption_features.npy", caption_test)

np.save("train_caption_to_image.npy", caption_to_train_img)
np.save("val_caption_to_image.npy", caption_to_val_img)
np.save("test_caption_to_image.npy", caption_to_test_img)

## Data Preparation Complete!

The following .npy files have been saved:

**Full dataset:**
- `flickr8k_image_features.npy`
- `flickr8k_caption_features.npy`
- `flickr8k_caption_to_image.npy`
- `flickr8k_image_names.npy`

**Train/Val/Test splits:**
- `train_image_features.npy`, `train_caption_features.npy`, `train_caption_to_image.npy`
- `val_image_features.npy`, `val_caption_features.npy`, `val_caption_to_image.npy`
- `test_image_features.npy`, `test_caption_features.npy`, `test_caption_to_image.npy`

These files can now be loaded by architecture notebooks (e.g., Corr_AE.ipynb, Cross_Modal_Autoencoder.ipynb).